## Task, Engine and Property Blocks



In [1]:
from scm.plams import Settings, AMSJob

To start with, in order to see the input that will be passed to AMS, create an AMSJob and print the input:

In [2]:
def print_input(settings):
    print(AMSJob(settings=settings).get_input())

Tasks can be specified under the `input.AMS.Task` key:

In [3]:
go_settings = Settings()
go_settings.input.AMS.Task = "GeometryOptimization"
go_settings.input.AMS.GeometryOptimization.Convergence.Gradients=1e-5
print_input(go_settings)

GeometryOptimization
  Convergence
    Gradients 1e-05
  End
End

Task GeometryOptimization




Properties can be specified under the `input.AMS.Properties` key:

In [4]:
nm_settings = Settings()
nm_settings.input.ams.Properties.NormalModes = "Yes"
print_input(nm_settings)

Properties
  NormalModes Yes
End




Engine settings can be specified under the `input.AMS.<Engine>` key, for the engine of interest:

In [5]:
lda_settings = Settings()
lda_settings.input.ADF.Basis.Type = "DZP"
print_input(lda_settings)


Engine ADF
  Basis
    Type DZP
  End
EndEngine




## Combining Settings

Settings objects can be combined for easy reuse and job setup.
Settings can be merged using the `+` and `+=` operators.

In [6]:
settings = go_settings + lda_settings + nm_settings
print_input(settings)

GeometryOptimization
  Convergence
    Gradients 1e-05
  End
End

Properties
  NormalModes Yes
End

Task GeometryOptimization


Engine ADF
  Basis
    Type DZP
  End
EndEngine




Note however that this is a "soft" update, so values of existing keys will not be overwritten:

In [7]:
pbe_settings = Settings()
pbe_settings.input.ADF.Basis.Type = "TZP"
pbe_settings.input.ADF.xc.gga="pbe"   
settings += pbe_settings
print_input(settings)

GeometryOptimization
  Convergence
    Gradients 1e-05
  End
End

Properties
  NormalModes Yes
End

Task GeometryOptimization


Engine ADF
  Basis
    Type DZP
  End
  xc
    gga pbe
  End
EndEngine




To achieve this behaviour, the `update` method can be used, which overwrites existing keys:

In [8]:
settings.update(pbe_settings)
print_input(settings)

GeometryOptimization
  Convergence
    Gradients 1e-05
  End
End

Properties
  NormalModes Yes
End

Task GeometryOptimization


Engine ADF
  Basis
    Type TZP
  End
  xc
    gga pbe
  End
EndEngine




Settings can also be removed using the `-` and `-=` operators:

In [9]:
settings -= nm_settings
print_input(settings)

Multiple values in a settings block can be configured using a list:

In [10]:
hybrid_settings = go_settings.copy()
hybrid_settings.input.AMS.Hybrid.Energy.Term = []
for i in range(5):
    factor = (-1) ** (i%2) * 1.0
    region = "*" if i == 0 else "one" if i < 3 else "two"
    engine_id = "adf-lda" if i == 0 or factor == -1 else "adf-gga"
    term = Settings({"Factor": factor, "Region": region, "EngineID": engine_id})
    hybrid_settings.input.AMS.Hybrid.Energy.Term.append(term)
hybrid_settings.input.AMS.Hybrid.Engine = [lda_settings.input.ADF.copy(), pbe_settings.input.ADF.copy()]
hybrid_settings.input.AMS.Hybrid.Engine[0]._h = "ADF adf-lda" 
hybrid_settings.input.AMS.Hybrid.Engine[1]._h = "ADF adf-gga" 
print_input(hybrid_settings)

GeometryOptimization
  Convergence
    Gradients 1e-05
  End
End

Hybrid
  Energy
    Term
      EngineID adf-lda
      Factor 1.0
      Region *
    End
    Term
      EngineID adf-lda
      Factor -1.0
      Region one
    End
    Term
      EngineID adf-gga
      Factor 1.0
      Region one
    End
    Term
      EngineID adf-lda
      Factor -1.0
      Region two
    End
    Term
      EngineID adf-gga
      Factor 1.0
      Region two
    End
  End
  Engine ADF adf-lda
    Basis
      Type DZP
    End
  EndEngine
  Engine ADF adf-gga
    Basis
      Type TZP
    End
    xc
      gga pbe
    End
  EndEngine

End

Task GeometryOptimization




Note also in the example below, the use of the special `_h` "header" key, which can be used to add data to the header line for a block.

## Nested Keys

It can be useful to access values from a Settings object using "nested" keys. These are tuples of keys, where each successive element of the tuple corresponds to a further layer in the settings. Lists are flattened so their elements can be accessed with the corresponding index.

In [11]:
list(hybrid_settings.nested_keys())

[('input',),
 ('input', 'AMS'),
 ('input', 'AMS', 'Task'),
 ('input', 'AMS', 'GeometryOptimization'),
 ('input', 'AMS', 'GeometryOptimization', 'Convergence'),
 ('input', 'AMS', 'GeometryOptimization', 'Convergence', 'Gradients'),
 ('input', 'AMS', 'Hybrid'),
 ('input', 'AMS', 'Hybrid', 'Energy'),
 ('input', 'AMS', 'Hybrid', 'Energy', 'Term'),
 ('input', 'AMS', 'Hybrid', 'Energy', 'Term', 0),
 ('input', 'AMS', 'Hybrid', 'Energy', 'Term', 0, 'Factor'),
 ('input', 'AMS', 'Hybrid', 'Energy', 'Term', 0, 'Region'),
 ('input', 'AMS', 'Hybrid', 'Energy', 'Term', 0, 'EngineID'),
 ('input', 'AMS', 'Hybrid', 'Energy', 'Term', 1),
 ('input', 'AMS', 'Hybrid', 'Energy', 'Term', 1, 'Factor'),
 ('input', 'AMS', 'Hybrid', 'Energy', 'Term', 1, 'Region'),
 ('input', 'AMS', 'Hybrid', 'Energy', 'Term', 1, 'EngineID'),
 ('input', 'AMS', 'Hybrid', 'Energy', 'Term', 2),
 ('input', 'AMS', 'Hybrid', 'Energy', 'Term', 2, 'Factor'),
 ('input', 'AMS', 'Hybrid', 'Energy', 'Term', 2, 'Region'),
 ('input', 'AMS', 'H

In [12]:
hybrid_settings.get_nested(("input", "AMS", "Task"))

'GeometryOptimization'

In [13]:
if hybrid_settings.contains_nested(("input", "AMS", "Hybrid", "Engine", 0)):
    hybrid_settings.set_nested(("input", "AMS", "Hybrid", "Engine", 0, "Basis", "Type"), "TZP")
print(hybrid_settings.get_nested(("input", "AMS", "Hybrid", "Engine", 0, "Basis")))

Type: 	TZP



## Comparison

Two settings objects can be compared to check the differences between them. The result will show the nested key and value of any added, removed and modified entries.

In [14]:
settings1 = go_settings + lda_settings + nm_settings
settings2 = go_settings.copy()
settings2.input.AMS.Task = "SinglePoint"
settings2.input.DFTB.Model = "GFN1-xTB"
print(settings2.compare(settings1))

{'added': {('input', 'DFTB', 'Model'): 'GFN1-xTB'}, 'removed': {('input', 'ADF', 'Basis', 'Type'): 'DZP', ('input', 'AMS', 'Properties', 'NormalModes'): 'Yes'}, 'modified': {('input', 'AMS', 'Task'): ('SinglePoint', 'GeometryOptimization')}}
